# Beta Weighting on a Portfolio / Thomas MEDARD

In [1]:
pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime as dt
import pandas as pd
import numpy as np
from scipy import stats
from pandas_datareader import data as pdr

#### Step 1: Specify date range for analysis
Here we begin by creating start and end dates using pythons datetime module.

In [3]:
start = dt.datetime(2021, 1, 1)
end = dt.datetime.now()
start, end

(datetime.datetime(2021, 1, 1, 0, 0),
 datetime.datetime(2024, 9, 29, 16, 44, 5, 877211))

#### Step 2: Select the stocks/tickers we would like to analyse

In [4]:
stockList = ['BNP', 'SAN', 'AIR', 'FP', 'MC']  # Exemples : BNP Paribas, Sanofi, Airbus, TotalEnergies, LVMH
stocks = ['^FCHI'] + [i + '.PA' for i in stockList]  # ^FCHI représente l'indice CAC 40
stocks

['^FCHI', 'BNP.PA', 'SAN.PA', 'AIR.PA', 'FP.PA', 'MC.PA']

#### Step 3 call the Yahoo Finance DataReader module:
using pdr.get_data_yahoo(stocks, start, end)

In [8]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

# Définir la liste des actions du CAC 40
stockList = ['BNP.PA', 'SAN.PA', 'AIR.PA', 'FP.PA', 'MC.PA']
stocks = ['^FCHI'] + stockList  # Ajouter l'indice CAC 40

# Définir la période de temps
end = datetime.now()
start = datetime(2018, 1, 1)

# Télécharger les données via yfinance
df = yf.download(stocks, start=start, end=end)

# Calculer les log-returns (logarithme des rendements)
log_returns = np.log(df['Close'] / df['Close'].shift(1)).dropna()

# Afficher les premières lignes des log-returns
log_returns.head()


[*********************100%***********************]  6 of 6 completed

1 Failed download:
['FP.PA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Ticker,AIR.PA,BNP.PA,FP.PA,MC.PA,SAN.PA,^FCHI
Date,,,,,,


#### Step 4a: Directly calculate beta:

$\frac{covariance(Market, Stock)}{variance(Market)}$

In [13]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

# Définir les actions du CAC 40 et l'indice du CAC 40
stockList = ['BNP.PA', 'SAN.PA', 'AIR.PA', 'FP.PA', 'MC.PA']
stocks = ['^FCHI'] + stockList  # ^FCHI est l'indice du CAC 40

# Définir la période
end = datetime.now()
start = datetime(2018, 1, 1)

# Télécharger les données via yfinance
df = yf.download(stocks, start=start, end=end)['Close']

# Calculer les rendements logarithmiques
log_returns = np.log(df / df.shift(1)).dropna()

# Fonction pour calculer le beta
def calc_beta(df):
    np_array = df.values  # Convertir le DataFrame en array numpy
    market_returns = np_array[:, 0]  # La première colonne est les rendements de l'indice de marché (CAC 40)
    beta = []
    
    # Calculer le beta pour chaque stock (excepté le marché lui-même)
    for ind, col in enumerate(df.columns):
        if ind > 0:  # Commencer à partir de la deuxième colonne (les actions)
            stock_returns = np_array[:, ind]
            # Calculer la matrice de covariance entre le marché et l'action
            covariance = np.cov(stock_returns, market_returns)
            # Beta = Cov(R_stock, R_market) / Var(R_market)
            beta_value = covariance[0, 1] / covariance[1, 1]
            beta.append(beta_value)
    
    return pd.Series(beta, df.columns[1:], name='Beta')  # Retourner les beta sous forme de série Pandas

# Calculer les betas pour les actions par rapport au CAC 40
betas = calc_beta(log_returns)
print(betas)


[*********************100%***********************]  6 of 6 completed

1 Failed download:
['FP.PA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Ticker
BNP.PA   NaN
FP.PA    NaN
MC.PA    NaN
SAN.PA   NaN
^FCHI    NaN
Name: Beta, dtype: float64


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/_core/_methods.py:139: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/var/folders/__/q1p_h225245503htypxk941c0000gn/T/ipykernel_10193/2226067564.py:31: RuntimeWarning: Degrees of freedom <= 0 for slice
  covariance = np.cov(stock_returns, market_returns)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2848: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2848: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [11]:
calc_beta(log_returns)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/_core/_methods.py:139: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/var/folders/__/q1p_h225245503htypxk941c0000gn/T/ipykernel_10193/1261897790.py:11: RuntimeWarning: Degrees of freedom <= 0 for slice
  covariance = np.cov(s,m)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2848: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2848: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Ticker
BNP.PA   NaN
FP.PA    NaN
MC.PA    NaN
SAN.PA   NaN
^FCHI    NaN
Name: Beta, dtype: float64

#### Step 4b: Use linear regression to get coefficient of market and stocks returns

In [ ]:
def regression_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            s = np_array[:,ind] # stock returns are column one from numpy array
            beta.append( stats.linregress(m,s)[0] )
    return pd.Series(beta, df.columns[1:], name='Beta')

In [ ]:
regression_beta(log_returns)

Symbols
CBA.AX    1.154583
NAB.AX    0.951789
WBC.AX    0.885018
ANZ.AX    0.905474
WPL.AX    1.150844
Name: Beta, dtype: float64

#### Step 4c: Use Matrix Algebra to complete linear regression in one line

For linear regression on a model of the form y=Xβ, where X is a matrix with full column rank, the least squares solution,

$\hat{\beta} = arg \min ||X\beta−y||_2 $


$\hat{\beta} = (X^T X)^{−1}X^Ty $

https://stats.stackexchange.com/questions/23128/solving-for-regression-parameters-in-closed-form-vs-gradient-descent/23132#23132

In [ ]:
def matrix_beta(df):
    # Market index is the first column 0
    X = df.values[:, [0]]
    # add an additional column for the intercept (initalise as 1's)
    X = np.concatenate([np.ones_like(X), X], axis=1)
    # Apply matrix algebra for linear regression model
    beta = np.linalg.pinv(X.T @ X) @ X.T @ df.values[:, 1:]
    return pd.Series(beta[1], df.columns[1:], name='Beta')

In [ ]:
beta = matrix_beta(log_returns)
beta

Symbols
CBA.AX    1.154583
NAB.AX    0.951789
WBC.AX    0.885018
ANZ.AX    0.905474
WPL.AX    1.150844
Name: Beta, dtype: float64

#### Step 5: Define your Portfolio and make DataFrame

Calculate Beta Weighted Portfolio

In [ ]:
units = np.array([100, 250, 300, 400, 200])
ASXprices = df.Close[-1:].values.tolist()[0]
price = np.array([round(price,2) for price in ASXprices[1:]])
value = [unit*pr for unit, pr in zip(units, price)]
weight = [round(val/sum(value),2) for val in value]
beta = round(beta,2)

In [ ]:
Portfolio = pd.DataFrame({
    'Stock': stockList,
    'Direction': 'Long',
    'Type': 'S',
    'Stock Price': price,
    'Price': price,
    'Units': units,
    'Value': units*price,
    'Weight': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Weight,Beta,Weighted Beta
Symbols,,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.04,100,10004.0,0.24,1.15,0.2760
NAB.AX,NAB,Long,S,30.43,30.43,250,7607.5,0.18,0.95,0.1710
WBC.AX,WBC,Long,S,23.88,23.88,300,7164.0,0.17,0.89,0.1513
ANZ.AX,ANZ,Long,S,28.20,28.20,400,11280.0,0.27,0.91,0.2457
WPL.AX,WPL,Long,S,27.43,27.43,200,5486.0,0.13,1.15,0.1495


#### Step 6: What if we have options, let's consider things in terms of Delta

In [ ]:
Portfolio = Portfolio.drop(['Weight', 'Weighted Beta'], axis=1)
Portfolio['Delta'] = Portfolio['Units']
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Symbols,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.04,100,10004.0,1.15,100
NAB.AX,NAB,Long,S,30.43,30.43,250,7607.5,0.95,250
WBC.AX,WBC,Long,S,23.88,23.88,300,7164.0,0.89,300
ANZ.AX,ANZ,Long,S,28.20,28.20,400,11280.0,0.91,400
WPL.AX,WPL,Long,S,27.43,27.43,200,5486.0,1.15,200


#### Add Options to portfolio
This is Only an example

In [ ]:
Options = [{'option':'CBA0Z8', 'underlying':'CBA', 'price':3.950, 'units': 2, 'delta': 0.627, 'direction': 'Short', 'type': 'Call'},
           {'option':'WPLQB9', 'underlying':'WPL', 'price':1.325, 'units': 2, 'delta': -0.425 ,'direction': 'Long', 'type': 'Put'}]

In [ ]:
for index, row in enumerate(Options):
    Portfolio.loc[row['option']] = [row['underlying'], row['direction'], row['type'], Portfolio.loc[row['underlying']+'.AX', 'Price'],
                                    row['price'], row['units'], row['price']*row['units']*100, beta[row['underlying']+'.AX'],
                                    (row['delta']*row['units']* 100 if row['direction'] == 'Long' else -row['delta']*row['units']*100)]
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Symbols,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.040,100,10004.0,1.15,100.0
NAB.AX,NAB,Long,S,30.43,30.430,250,7607.5,0.95,250.0
WBC.AX,WBC,Long,S,23.88,23.880,300,7164.0,0.89,300.0
ANZ.AX,ANZ,Long,S,28.20,28.200,400,11280.0,0.91,400.0
WPL.AX,WPL,Long,S,27.43,27.430,200,5486.0,1.15,200.0
CBA0Z8,CBA,Short,Call,100.04,3.950,2,790.0,1.15,-125.4
WPLQB9,WPL,Long,Put,27.43,1.325,2,265.0,1.15,-85.0


#### Step 7: Weight the Delta's using Beta

In [ ]:
Portfolio['ASX200 Weighted Delta (point)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']/ASXprices[0]) * Portfolio['Delta'],2)
Portfolio['ASX200 Weighted Delta (1%)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']) * Portfolio['Delta'] * 0.01,2)
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta,ASX200 Weighted Delta (point),ASX200 Weighted Delta (1%)
Symbols,,,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.040,100,10004.0,1.15,100.0,1.59,115.05
NAB.AX,NAB,Long,S,30.43,30.430,250,7607.5,0.95,250.0,1.00,72.27
WBC.AX,WBC,Long,S,23.88,23.880,300,7164.0,0.89,300.0,0.88,63.76
ANZ.AX,ANZ,Long,S,28.20,28.200,400,11280.0,0.91,400.0,1.42,102.65
WPL.AX,WPL,Long,S,27.43,27.430,200,5486.0,1.15,200.0,0.87,63.09
CBA0Z8,CBA,Short,Call,100.04,3.950,2,790.0,1.15,-125.4,-1.99,-144.27
WPLQB9,WPL,Long,Put,27.43,1.325,2,265.0,1.15,-85.0,-0.37,-26.81


#### Step 8: Total the Delta's to get Portfolio Overview

In [ ]:
Portfolio.loc['Total', ['Value', 'ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']] \
= Portfolio[['Value','ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']].sum()
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta,ASX200 Weighted Delta (point),ASX200 Weighted Delta (1%)
Symbols,,,,,,,,,,,
CBA.AX,CBA,Long,S,100.04,100.040,100.0,10004.0,1.15,100.0,1.59,115.05
NAB.AX,NAB,Long,S,30.43,30.430,250.0,7607.5,0.95,250.0,1.00,72.27
WBC.AX,WBC,Long,S,23.88,23.880,300.0,7164.0,0.89,300.0,0.88,63.76
ANZ.AX,ANZ,Long,S,28.20,28.200,400.0,11280.0,0.91,400.0,1.42,102.65
WPL.AX,WPL,Long,S,27.43,27.430,200.0,5486.0,1.15,200.0,0.87,63.09
CBA0Z8,CBA,Short,Call,100.04,3.950,2.0,790.0,1.15,-125.4,-1.99,-144.27
WPLQB9,WPL,Long,Put,27.43,1.325,2.0,265.0,1.15,-85.0,-0.37,-26.81
Total,NaN,NaN,NaN,NaN,NaN,NaN,42596.5,NaN,NaN,3.40,245.74
